In [1]:
%%capture
!{sys.executable} -m pip install openai
!{sys.executable} -m pip install litellm

In [2]:
import os
from openai import OpenAI

if not os.environ["OPENAI_API_KEY"]: 
    os.environ["OPENAI_API_KEY"]= '<REDACTED>'

MODEL = "gpt-4o-mini"

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

Open keywords data:

In [3]:
def read_file_to_list(file_path):
    with open(file_path, 'r') as file:
        lines = file.read().splitlines()
    return lines

keywords_list = read_file_to_list('20qs-data/keywords.txt')
keywords_list = keywords_list[1:]
print(f"Number of keywords in data: {len(keywords_list)}")

Number of keywords in data: 2046


## Labeling Functions

In [4]:
from collections import Counter
from tqdm import tqdm

def get_answer(label, keyword):
    
    prompt = f"Keyword: {keyword}\n\nQuestion: {label['question']}\n\n"
    prompt += "Please provide an answer to the question based on the keyword. ONLY ANSWER Yes OR No. IF UNSURE, CHOOSE MOST LIKELY ANSWER FROM Yes OR No. "
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=10
    )
    response = response.choices[0].message.content

    return response


def label_keywords(label, keywords, labels, results):
    
    label_tag = label['tag']
    yes_no_counts = Counter()

    for keyword in tqdm(keywords, desc = 'label_keywords', position = 0, leave = True, ncols = 100):
        answer = get_answer(label, keyword)
        answer = answer.lower()
        # Count yes/no answers
        if 'yes' in answer:
            yes_no_counts['yes'] += 1
            answer = 'yes'
        elif 'no' in answer:
            yes_no_counts['no'] += 1
            answer = 'no'
        else:
            yes_no_counts['err'] +=1
            answer = 'err'
        if keyword in labels:
            labels[keyword][label_tag] = answer
        else:
            labels[keyword] = {label_tag: answer}

    # Calculate percentages
    total_answers = sum(yes_no_counts.values())
    yes_percentage = (yes_no_counts['yes'] / total_answers) * 100 if total_answers > 0 else 0
    no_percentage = (yes_no_counts['no'] / total_answers) * 100 if total_answers > 0 else 0
    err_percentage = (yes_no_counts['err'] / total_answers) * 100 if total_answers > 0 else 0

    # Print results
    print(label['question'])
    print(f"Yes answers: {yes_no_counts['yes']}, percentage: {yes_percentage:.2f}%")
    print(f"No answers: {yes_no_counts['no']}, percentage: {no_percentage:.2f}%")
    print(f"Error answers: {yes_no_counts['err']}, percentage: {err_percentage:.2f}%")

    # Save results
    results[label_tag] = {
        'yes_percentage': yes_percentage,
        'no_percentage': no_percentage,
        'yes_counts': yes_no_counts['yes'],
        'no_counts': yes_no_counts['no'],
        'err_counts': yes_no_counts['err']
    }

    return labels, results

Read current keyword labels and results for adding labels:

In [7]:
import json

def read_jsonl_and_transform(file_path):
    result_dict = {}

    with open(file_path, 'r') as file:
        for line in file:
            record = json.loads(line)
            keyword = record.pop('keyword')
            result_dict[keyword] = record

    return result_dict

def read_json_to_dict(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

keyword_labels, results = read_jsonl_and_transform('20qs-data/labeled_keywords-v2.jsonl'), read_json_to_dict('20qs-data/labeling_results-v2.jsonl')

OR start without any labels:

In [13]:
#keyword_labels = {}
#results = {}

Label places/things:

In [14]:
places_label = {'tag': 'place', 'question': "Is it a place?"}

keyword_labels, results = label_keywords(places_label, keywords_list, keyword_labels, results)

label_keywords:   0%|                                                      | 0/2046 [00:00<?, ?it/s]

label_keywords: 100%|███████████████████████████████████████████| 2046/2046 [17:45<00:00,  1.92it/s]

Is it a place?
Yes answers: 699, percentage: 34.16%
No answers: 1347, percentage: 65.84%
Error answers: 0, percentage: 0.00%


In [8]:
keyword_places = [keyword for keyword in keywords_list if keyword_labels[keyword]['place'] == 'yes']
keyword_things = [keyword for keyword in keywords_list if keyword_labels[keyword]['place'] == 'no']

print(f"Number of keywords labeled as things: {len(keyword_things)}.")

Number of keywords labeled as things: 1347.


## Things Branch

Labels to apply:

In [16]:
labels_things = [
    {'tag': 'food', 'question': "Is it a food?"},
    {'tag': 'beverage', 'question': "Is it a beverage?"},
    {'tag': 'living', 'question': "Is it a living thing?"},
    {'tag': 'plant', 'question': "Is it a plant?"},
    {'tag': 'animal', 'question': 'Is it an animal?'},
    {'tag': 'electronics', 'question': "Is it related to electronics or technology?"},
    {'tag': 'furniture', 'question': 'Is it furniture?'},
    {'tag': 'transportation, vehicles', 'question': 'Is it related to transportation or vehicles?'},
    {'tag': 'man-made', 'question': "Is it a man-made thing?"},
    {'tag': 'furniture', 'question': "Is it furniture?"},
    {'tag': 'architecture, construction', 'question': 'Is it related to architectural structures or construction?'},
    {'tag': 'natural material, resource', 'question': 'Is it a natural material or resource?'},
    {'tag': 'natural phenomenon', 'question': 'Is it a natural phenomenon?'},
    {'tag': 'industry, manufacturing', 'question': "Is it related to industrial production or manufacturing?"},
    {'tag': 'agriculture', 'question': "Is it related to agricultural production?"},
    {'tag': 'arts, media', 'question': "Is it broadly related to the arts or media?"},
    {'tag': 'safety', 'question': "Is it related to safety or safety equipments?"},
    {'tag': 'medicine', 'question': "Is it broadly related to medicine or healthcare?"},
    {'tag': 'clothing, accessories, beauty', 'question': "Is it related to clothing, accessories or beauty products?"},
    {'tag': 'sports, entertainment', 'question': "Is it broadly related to entertainment or sports?"}
]

In [17]:
for label in tqdm(labels_things, desc = 'labels', ncols=150):
    keyword_labels, results = label_keywords(label, keyword_things, keyword_labels, results)

labels:   5%|█████▏                                                                                                 | 1/20 [11:38<3:41:17, 698.80s/it]

Is it a food?
Yes answers: 239, percentage: 17.74%
No answers: 1108, percentage: 82.26%
Error answers: 0, percentage: 0.00%


labels:  10%|██████████▎                                                                                            | 2/20 [23:11<3:28:38, 695.50s/it]

Is it a beverage?
Yes answers: 46, percentage: 3.41%
No answers: 1301, percentage: 96.59%
Error answers: 0, percentage: 0.00%


labels:  15%|███████████████▍                                                                                       | 3/20 [34:15<3:12:51, 680.70s/it]

...

Is it broadly related to medicine or healthcare?
Yes answers: 281, percentage: 20.86%
No answers: 1066, percentage: 79.14%
Error answers: 0, percentage: 0.00%


labels:  95%|████████████████████████████████████████████████████████████████████████████████████████████████▉     | 19/20 [3:36:33<11:23, 683.21s/it]

Is it related to clothing, accessories or beauty products?
Yes answers: 249, percentage: 18.49%
No answers: 1098, percentage: 81.51%
Error answers: 0, percentage: 0.00%


labels: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [3:48:02<00:00, 684.10s/it]

Is it broadly related to entertainment or sports?
Yes answers: 281, percentage: 20.86%
No answers: 1066, percentage: 79.14%
Error answers: 0, percentage: 0.00%


In [9]:
labels_things_to_add_or_modify = [
    {'tag': 'food, beverage, cooking', 'question': 'Is it related to food, beverages or cooking?'},
    {'tag': 'handheld', 'question': 'Is it something a person can hold in their hand?'},
    {'tag': 'industry, manufacturing', 'question': "Is it used in industrial production or manufacturing?"},
    {'tag': 'agriculture', 'question': "Is it used in agricultural production?"},
    {'tag': 'transportation, vehicles', 'question': 'Is it related to transportation or vehicles?'},
    {'tag': 'indoors', 'question': 'Is it something that can be found indoors?'}
]

In [10]:
for label in tqdm(labels_things_to_add_or_modify, desc = 'labels', ncols=150):
    keyword_labels, results = label_keywords(label, keyword_things, keyword_labels, results)

labels:  17%|█████████████████▋                                                                                        | 1/6 [10:50<54:14, 650.84s/it]

Is it related to food, beverages or cooking?
Yes answers: 410, percentage: 30.44%
No answers: 937, percentage: 69.56%
Error answers: 0, percentage: 0.00%


labels:  33%|███████████████████████████████████▎                                                                      | 2/6 [21:39<43:18, 649.57s/it]

Is it something a person can hold in their hand?
Yes answers: 941, percentage: 69.86%
No answers: 406, percentage: 30.14%
Error answers: 0, percentage: 0.00%


labels:  50%|█████████████████████████████████████████████████████                                                     | 3/6 [32:37<32:39, 653.25s/it]

...

Is it used in agricultural production?
Yes answers: 556, percentage: 41.28%
No answers: 791, percentage: 58.72%
Error answers: 0, percentage: 0.00%


labels:  83%|████████████████████████████████████████████████████████████████████████████████████████▎                 | 5/6 [54:16<10:51, 651.01s/it]

Is it related to transportation or vehicles?
Yes answers: 169, percentage: 12.55%
No answers: 1178, percentage: 87.45%
Error answers: 0, percentage: 0.00%


labels: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [1:05:23<00:00, 653.92s/it]

Is it something that can be found indoors?
Yes answers: 1133, percentage: 84.11%
No answers: 214, percentage: 15.89%
Error answers: 0, percentage: 0.00%


## Places Branch
Out of use as of the last competition update.

In [18]:
labels_places = [
    {'tag': 'country', 'question': "Is it a country?"},
    {'tag': 'city', 'question': "Is it a city?"},
    {'tag': 'capital', 'question': "Is it a capital of a country?"},
    {'tag': 'landlocked', 'question': "Is the place either a landlocked country or a place located in a landlocked country?"},
    {'tag': 'coast', 'question': 'Is this place found along the coast?'},
    {'tag': 'geological feature', 'question': 'Is it a geological feature?'},
    {'tag': 'africa', 'question': "Is it in Africa?"},
    {'tag': 'asia', 'question': 'Is it in Asia?'},
    {'tag': 'europe', 'question': 'Is it in Europe?'},
    {'tag': 'hemisphere', 'question': 'Is it in the Western hemisphere?'},
    {'tag': 'oceania', 'question': 'Is it in Oceania?'},
    {'tag': 'europe or asia', 'question': "Is it either in europe or asia?"},
    {'tag': 'americas', 'question': "Is it in the Americas?"},
    {'tag': 'north america', 'question': 'Is it in North America?'},
    {'tag': 'man-made landmark', 'question': 'Is it a man-made landmark?'},
    {'tag': 'natural landmark', 'question': 'Is it a natural landmark?'},
    {'tag': 'equator', 'question': "Is it north of the equator?"},
    {'tag': 'body of water', 'question': "Is it a body of water?"},
    {'tag': 'biome', 'question': "Is it a type of biome?"},
    {'tag': 'building', 'question': "Is it a type of building or stucture?"}
]

In [19]:
for label in tqdm(labels_places, desc = 'labels', ncols=150):
    keyword_labels, results = label_keywords(label, keyword_places, keyword_labels, results)

labels:   5%|█████▏                                                                                                 | 1/20 [06:20<2:00:33, 380.74s/it]

Is it a country?
Yes answers: 194, percentage: 27.75%
No answers: 505, percentage: 72.25%
Error answers: 0, percentage: 0.00%


labels:  10%|██████████▎                                                                                            | 2/20 [12:29<1:52:09, 373.88s/it]

Is it a city?
Yes answers: 307, percentage: 43.92%
No answers: 392, percentage: 56.08%
Error answers: 0, percentage: 0.00%


labels:  15%|███████████████▍                                                                                       | 3/20 [18:31<1:44:22, 368.36s/it]

...

Is it a body of water?
Yes answers: 20, percentage: 2.86%
No answers: 679, percentage: 97.14%
Error answers: 0, percentage: 0.00%


labels:  95%|████████████████████████████████████████████████████████████████████████████████████████████████▉     | 19/20 [1:55:58<06:04, 364.72s/it]

Is it a type of biome?
Yes answers: 8, percentage: 1.14%
No answers: 691, percentage: 98.86%
Error answers: 0, percentage: 0.00%


labels: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [2:01:53<00:00, 365.67s/it]

Is it a type of building or stucture?
Yes answers: 50, percentage: 7.15%
No answers: 649, percentage: 92.85%
Error answers: 0, percentage: 0.00%


## Save as JSONL

In [11]:
def make_list_from_dict(original_dict):
    list_of_dicts = []
    for key, sub_dict in original_dict.items():
        # Create a new dictionary that includes the key from the original dict
        new_dict = {"keyword": key}
        # Update the new dictionary with the key-value pairs from the sub-dictionary
        new_dict.update(sub_dict)
        # Append the new dictionary to the list
        list_of_dicts.append(new_dict)
    return list_of_dicts

labeled_keywords_list = make_list_from_dict(keyword_labels)

In [12]:
with open("20qs-data/labeled_keywords_v2.jsonl", "w") as f:
    for i in labeled_keywords_list:
        json.dump(i, f)
        f.write('\n')

with open("20qs-data/labeling_results_v2.jsonl", "w") as f:
    json.dump(results, f)